In [14]:
from langchain.tools.render import render_text_description # to describe tools as a string 
from langchain_core.tools import tool # tools for our llm

import logging
logging.basicConfig(level=logging.INFO)


In [15]:

tools_logger = logging.getLogger("tools")
@tool
def accept(reason) -> int:
    "Accept a reason and return 1."
    tools_logger.info(f"Accepting reason: {reason}")
    return 1


@tool
def reject(reason) -> int:
    "Reject a reason and return 0."
    tools_logger.info(f"Rejecting reason: {reason}")
    return 0

In [16]:
tools = [accept,reject]
rendered_tools = render_text_description(tools)
print(rendered_tools)

accept: accept(reason) -> int - Accept a reason and return 1.
reject: reject(reason) -> int - Reject a reason and return 0.


In [17]:
system_prompt = f"""You are an assistant that has access to functions.
Use them if required:

{rendered_tools}
Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.
The value associated with the 'arguments' key should be a dictionary of parameters."""
system_prompt

"You are an assistant that has access to functions.\nUse them if required:\n\naccept: accept(reason) -> int - Accept a reason and return 1.\nreject: reject(reason) -> int - Reject a reason and return 0.\nGiven the user input, return the name and input of the tool to use.\nReturn your response as a JSON blob with 'name' and 'arguments' keys.\nThe value associated with the 'arguments' key should be a dictionary of parameters."

In [18]:
import requests
from datetime import datetime
def LLM(prompt, system_message, temperature, max_tokens,top_p, model,return_json=False):
    system_message = "Todays DateTime:"+datetime.now().strftime("%Y-%m-%d %H:00:00")+"\n"+system_message
    url = 'http://localhost:11435/api/passthrough/chat'
    headers = {'Content-Type': 'application/json'}
    payload = {
        'model': model,
        'cache': '1h',
        'keep_alive': '0m',
        'messages': [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': prompt}
        ],
        'temperature': temperature,
        'top_p': top_p,
        'max_tokens': max_tokens,
        'return_json': return_json
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an HTTPError if the HTTP request returned an unsuccessful status code
        data = response.json()
        #print('Success:', data)
        return data
    except requests.exceptions.RequestException as error:
        print('Error:', error)
        raise


In [22]:
LLM("""
    You are a video game bot.
    I am being traded:
    100GP
    I am aquiring:
    Cow (10gp)
    Only Use the provided tools.
    
    
    
   """,system_prompt,temperature=0.5, top_p=0.65,model="llama3-groq-tool-use",max_tokens=9999,return_json=False)

TypeError: LLM() missing 1 required positional argument: 'max_tokens'

In [ ]:
LLM("""
    what is the weather in San Francisco?
    ""","""
    
    
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:
<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "properties": {
            "location": {
                "description": "The city and state, e.g. San Francisco, CA",
                "type": "string"
            },
            "unit": {
                "enum": [
                    "celsius",
                    "fahrenheit"
                ],
                "type": "string"
            }
        },
        "required": [
            "location"
        ],
        "type": "object"
    }
} </tools>

""",temperature=0.5, top_p=0.65 ,model="llama3-groq-tool-use",return_json=False)

TypeError: LLM() got an unexpected keyword argument 'top_p'